In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
# โหลดโมเดลและ Tokenizer
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

# Test Sliding windows

## Test without dynamic

In [ ]:
long_text2 = """ The Role of Artificial Intelligence in Modern Healthcare
Artificial intelligence (AI) is transforming numerous industries, and healthcare stands as one of the most significantly impacted fields.
With the ever-increasing amounts of data generated in healthcare—from patient records and diagnostic images to genomic sequences—AI technologies offer powerful tools for analysis,
prediction, and decision-making. By leveraging machine learning algorithms, natural language processing (NLP), and computer vision, AI applications are enhancing the accuracy of diagnoses,
optimizing treatment plans, and improving patient outcomes.
AI in Diagnostics
One of the most critical applications of AI in healthcare is in diagnostics. Machine learning models trained on vast datasets of medical images can detect abnormalities such as tumors, fractures,
or signs of diseases like diabetic retinopathy. For instance, AI algorithms in radiology can analyze X-rays, CT scans, and MRIs with remarkable speed and accuracy, often surpassing human performance.
Similarly, in pathology, AI systems can assist in identifying cancerous cells from histopathological slides, reducing the risk of human error and enabling faster diagnoses.
Personalized Medicine
Personalized medicine, a growing trend in healthcare, has also benefited immensely from AI. By analyzing genomic data, AI can identify genetic markers linked to specific diseases and suggest tailored treatment
plans. This capability is particularly significant in oncology, where understanding a tumor's genetic profile can guide the selection of targeted therapies, maximizing efficacy while minimizing adverse effects.
Furthermore, AI-driven tools enable predictive analytics, which can assess a patient’s risk of developing certain conditions based on their genetic and lifestyle factors, enabling early intervention.
Operational Efficiency
AI is not limited to clinical applications; it also plays a vital role in improving operational efficiency within healthcare systems. Predictive analytics can optimize resource allocation,
such as predicting patient admission rates to manage hospital capacity effectively. Chatbots and virtual assistants powered by NLP can handle routine inquiries, schedule appointments,
and provide preliminary medical advice, freeing up healthcare professionals to focus on more complex tasks. Additionally, robotic process automation (RPA) can streamline administrative processes,
such as billing and claims processing, reducing errors and cutting costs.
Ethical Considerations
Despite its immense potential, the integration of AI in healthcare raises several ethical concerns. Issues surrounding patient privacy, data security, and algorithmic bias require careful consideration.
Ensuring that AI models are trained on diverse and representative datasets is critical to avoid perpetuating health disparities. Moreover, transparent and explainable AI systems are essential to build
trust among clinicians and patients. Regulatory frameworks and interdisciplinary collaboration will play a key role in addressing these challenges, ensuring that AI technologies are implemented responsibly.
Future Directions
The future of AI in healthcare is promising, with emerging technologies such as federated learning, which allows AI models to be trained across decentralized data sources without compromising privacy.
Innovations in wearable devices and Internet of Medical Things (IoMT) are expected to further enhance remote monitoring and telemedicine services. Additionally, integrating AI with other cutting-edge
technologies, such as blockchain for secure data sharing and quantum computing for complex problem-solving, could unlock new possibilities in the healthcare landscape.
"""

In [ ]:
summary_2 = """ AI is revolutionizing healthcare by enhancing diagnostics, enabling personalized medicine, and improving operational efficiency.
While challenges related to ethics, privacy, and bias remain, advancements like federated learning and IoMT promise a future of safer, more efficient, and more equitable healthcare solutions."""

In [ ]:
tokens2 = tokenizer.encode(long_text2, return_tensors="pt", truncation=False)
tokens2.size(1)

697

In [ ]:
def sliding_window_summarization1(text, max_length=509, overlap=128, prefix = "summarize: "):

    tokens = tokenizer.encode(text, return_tensors="pt", truncation=False)
    token_chunks = []
    start = 0

    # สร้าง chunks ที่มีการทับซ้อน
    while start < tokens.size(1):
        end = start + max_length
        chunk_tokens = tokens[:, start:end]
        chunk_text = tokenizer.decode(chunk_tokens[0], skip_special_tokens=True)
        # เพิ่ม prefix
        chunk_with_prefix = prefix + chunk_text
        token_chunks.append(chunk_with_prefix)
        start = end - overlap  # เลื่อนหน้าต่างโดยมีการทับซ้อน

    return token_chunks

In [ ]:
test1 = sliding_window_summarization1(long_text2)
test1

["summarize: The Role of Artificial Intelligence in Modern Healthcare Artificial intelligence (AI) is transforming numerous industries, and healthcare stands as one of the most significantly impacted fields. With the ever-increasing amounts of data generated in healthcare—from patient records and diagnostic images to genomic sequences—AI technologies offer powerful tools for analysis, prediction, and decision-making. By leveraging machine learning algorithms, natural language processing (NLP), and computer vision, AI applications are enhancing the accuracy of diagnoses, optimizing treatment plans, and improving patient outcomes. AI in Diagnostics One of the most critical applications of AI in healthcare is in diagnostics. Machine learning models trained on vast datasets of medical images can detect abnormalities such as tumors, fractures, or signs of diseases like diabetic retinopathy. For instance, AI algorithms in radiology can analyze X-rays, CT scans, and MRIs with remarkable speed

In [ ]:
test2 = [tokenizer.encode(chunk, return_tensors="pt", max_length=512, truncation=False) for chunk in test1]
test2

[tensor([[21603,    10,    37,  2158,   109,    13, 24714,  5869,  2825,  1433,
             16,  5070, 16254, 24714,  6123,    41,  9822,    61,    19,     3,
          21139,  2724,  5238,     6,    11,  4640,  5024,    38,    80,    13,
              8,   167,  4019,     3, 18229,  4120,     5,   438,     8,   664,
             18,    77,  5045,     9,     7,    53,  6201,    13,   331,  6126,
             16,  4640,   318,  7152,  1868,  3187,    11,  7028,  1383,    12,
          29082,  5932,     7,   318,  9822,  2896,   462,  2021,  1339,    21,
           1693,     6, 21332,     6,    11,  1357,    18,  5239,     5,   938,
              3, 26072,  1437,  1036, 16783,     6,   793,  1612,  3026,    41,
            567,  6892,   201,    11,  1218,  2267,     6,  7833,  1564,    33,
              3, 14762,     8,  7452,    13, 18730,     7,     6, 19769,    53,
           1058,  1390,     6,    11,  4863,  1868,  6353,     5,  7833,    16,
          27678,     7,   555,    13,   

In [ ]:
# ฟังก์ชัน Sliding Window
def sliding_window_summarization2(text, max_length=509, overlap=128, prefix = "summarize: "):

    tokens = tokenizer.encode(text, return_tensors="pt", truncation=False)
    token_chunks = []; summaries = []
    start = 0

    # สร้าง chunks ที่มีการทับซ้อน
    while start < tokens.size(1):
        end = start + max_length
        chunk_tokens = tokens[:, start:end]
        chunk_text = tokenizer.decode(chunk_tokens[0], skip_special_tokens=True)
        # เพิ่ม prefix
        chunk_with_prefix = prefix + chunk_text
        token_chunks.append(chunk_with_prefix)
        start = end - overlap  # เลื่อนหน้าต่างโดยมีการทับซ้อน

    for chunk in token_chunks:
      # Encode chunk เพื่อใช้กับโมเดล
      inputs = tokenizer.encode(chunk, return_tensors="pt", max_length=512, truncation=False)
      # สร้างสรุป
      summary_ids = model.generate(inputs, max_length=150, num_beams=4, early_stopping=True)
      # ถอดรหัสข้อความสรุป
      summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
      summaries.append(summary)

    final_summary = " ".join(summaries)
    return final_summary

In [ ]:
sliding_window_summarization2(long_text2)

'AI is transforming numerous industries, and healthcare stands as one of the most significantly impacted fields. by leveraging machine learning algorithms, natural language processing (NLP) and computer vision, AI applications are enhancing the accuracy of diagnoses, optimizing treatment plans, and improving patient outcomes. predictive analytics can optimize resource allocation, such as predicting patient admission rates to manage hospital capacity effectively. robotic process automation (RPA) can streamline administrative processes, such as billing and claims processing, reducing errors and cutting costs.'

In [ ]:
def test_function(examples, prefix = "summarize: "):

    inputs = tokenizer.encode(prefix + examples, return_tensors="pt", max_length=512, truncation=False)
    outputs = model.generate(inputs, max_length=150, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # model_inputs = tokenizer(inputs, max_length=512, truncation=False)
    # labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)
    # model_inputs["labels"] = labels["input_ids"]
    return inputs, outputs, summary

In [ ]:
test_function(sliding_window_summarization2(long_text2))

(tensor([[21603,    10,  7833,    19,     3, 21139,  2724,  5238,     6,    11,
           4640,  5024,    38,    80,    13,     8,   167,  4019,     3, 18229,
           4120,     5,    57,     3, 26072,  1437,  1036, 16783,     6,   793,
           1612,  3026,    41,   567,  6892,    61,    11,  1218,  2267,     6,
           7833,  1564,    33,     3, 14762,     8,  7452,    13, 18730,     7,
              6, 19769,    53,  1058,  1390,     6,    11,  4863,  1868,  6353,
              5, 27875,  9952,    54, 13436,  3487, 17127,     6,   224,    38,
              3, 29856,  1868,  7209,  1917,    12,  1865,  2833,  2614,  3762,
              5, 20407,   433, 11747,    41,  6294,   188,    61,    54, 24104,
           6601,  2842,     6,   224,    38, 14425,    11,  3213,  3026,     6,
              3,  5503,  6854,    11,  3753,  1358,     5,     1]]),
 tensor([[    0,  7833,  1564,    33,     3, 14762,     8,  7452,    13, 18730,
              7,     6, 19769,    53,  1058,  1390,

In [ ]:
summary_2

' AI is revolutionizing healthcare by enhancing diagnostics, enabling personalized medicine, and improving operational efficiency. \nWhile challenges related to ethics, privacy, and bias remain, advancements like federated learning and IoMT promise a future of safer, more efficient, and more equitable healthcare solutions.'

For using in future

In [ ]:
def preprocess_function(examples, prefix = "summarize: "):

    inputs = prefix + examples
    model_inputs = tokenizer(inputs, max_length=512, truncation=False)
    labels = tokenizer(text_target=summary_2, max_length=150, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
test3 = preprocess_function(sliding_window_summarization2(long_text2))
test3

{'input_ids': [21603, 10, 7833, 19, 3, 21139, 2724, 5238, 6, 11, 4640, 5024, 38, 80, 13, 8, 167, 4019, 3, 18229, 4120, 5, 57, 3, 26072, 1437, 1036, 16783, 6, 793, 1612, 3026, 41, 567, 6892, 61, 11, 1218, 2267, 6, 7833, 1564, 33, 3, 14762, 8, 7452, 13, 18730, 7, 6, 19769, 53, 1058, 1390, 6, 11, 4863, 1868, 6353, 5, 27875, 9952, 54, 13436, 3487, 17127, 6, 224, 38, 3, 29856, 1868, 7209, 1917, 12, 1865, 2833, 2614, 3762, 5, 20407, 433, 11747, 41, 6294, 188, 61, 54, 24104, 6601, 2842, 6, 224, 38, 14425, 11, 3213, 3026, 6, 3, 5503, 6854, 11, 3753, 1358, 5, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [7833, 19, 9481, 2610, 4640, 57, 3, 14762, 7028, 7, 6, 3, 10165, 9354, 4404, 6, 11, 4

In [ ]:
tokenizer.decode(test3['input_ids'], skip_special_tokens=True)

'summarize: AI is transforming numerous industries, and healthcare stands as one of the most significantly impacted fields. by leveraging machine learning algorithms, natural language processing (NLP) and computer vision, AI applications are enhancing the accuracy of diagnoses, optimizing treatment plans, and improving patient outcomes. predictive analytics can optimize resource allocation, such as predicting patient admission rates to manage hospital capacity effectively. robotic process automation (RPA) can streamline administrative processes, such as billing and claims processing, reducing errors and cutting costs.'

In [ ]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from evaluate import load

rouge = load("rouge")
# Define the summarized and reference texts
predictions = [""" AI applications are enhancing the accuracy of diagnoses, optimizing treatment plans,
           and improving patient outcomes. robotic process automation (RPA) can streamline
           administrative processes, such as billing and claims processing, reducing errors and cutting costs. """]

references = [summary_2]

results = rouge.compute(predictions=predictions, references=references)
print(results)

{'rouge1': 0.18666666666666668, 'rouge2': 0.027397260273972605, 'rougeL': 0.16, 'rougeLsum': 0.18666666666666668}


## With dynamic

In [ ]:
# ฟังก์ชัน Sliding Window
def sliding_window_summarization2(text, max_length=509, overlap=128, prefix = "summarize: "):

    tokens = tokenizer.encode(text, return_tensors="pt", truncation=False)
    token_chunks = []; summaries = []
    start = 0

    # สร้าง chunks ที่มีการทับซ้อน
    while start < tokens.size(1):
        end = start + max_length
        chunk_tokens = tokens[:, start:end]
        chunk_text = tokenizer.decode(chunk_tokens[0], skip_special_tokens=True)
        # เพิ่ม prefix
        chunk_with_prefix = prefix + chunk_text
        token_chunks.append(chunk_with_prefix)
        start = end - overlap  # เลื่อนหน้าต่างโดยมีการทับซ้อน

    for chunk in token_chunks:
      # Encode chunk เพื่อใช้กับโมเดล
      inputs = tokenizer.encode(chunk, return_tensors="pt", max_length=512, truncation=False)
      # สร้างสรุป
      summary_ids = model.generate(inputs, max_length=150, num_beams=4, early_stopping=True)
      # ถอดรหัสข้อความสรุป
      summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
      summaries.append(summary)

    final_summary = " ".join(summaries)
    return final_summary

In [ ]:
def preprocess_function(examples, prefix = "summarize: "):

    inputs = prefix + examples
    model_inputs = tokenizer(inputs, max_length=512, truncation=False)
    labels = tokenizer(text_target=summary_2, max_length=150, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Test Extractive summarization

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# แยกข้อความเป็นประโยค
sentences = long_text.split(". ")

# ใช้ TF-IDF เพื่อวิเคราะห์ความสำคัญของประโยค
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(sentences)

# คำนวณความคล้ายคลึง (Cosine Similarity)
scores = tfidf_matrix.sum(axis=1)
scores = np.array(scores).flatten()

# จัดอันดับประโยค
ranked_sentences = [sentences[i] for i in scores.argsort()[::-1]]

# แสดงผลสรุป (เลือก 2 ประโยคสำคัญ)
summary = ". ".join(ranked_sentences[:2])
print("Summary:", summary)


Summary: From Ethiopia, coffee could have been introduced to Yemen via trade across the Red Sea.
[14] One account credits Muhammad Ibn Sa'd for bringing the beverage to Aden from the African coast,[15] other early accounts say Ali ben Omar of the Shadhili Sufi order was the first to introduce coffee to Arabia.
[15][16] 16th-century Islamic scholar Ibn Hajar al-Haytami notes in his writings that a beverage called qahwa developed from a tree in the Zeila region located in the Horn of Africa.
[13] Coffee was first exported from Ethiopia to Yemen by Somali merchants from Berbera and Zeila in modern-day Somaliland, which was procured from Harar and the Abyssinian interior. The first plants from these smuggled seeds were planted in Mysore.
Coffee had spread to Italy by 1600 and then to the rest of Europe, Indonesia, and the Americas.[19]In 1583, Leonhard Rauwolf, a German physician, gave this description of coffee after returning from a ten-year trip
to the Near East: A beverage as black as 

In [ ]:
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# ข้อความต้นฉบับ
text = """
Coffee originated in Yemen in the 15th century. It was used by Sufi circles for religious rituals.
From Yemen, coffee spread to the Middle East, Europe, and eventually the Americas.
Today, coffee is one of the most popular beverages in the world.
"""

# แยกข้อความเป็นประโยค
sentences = text.split(". ")

# คำนวณ TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(sentences)

# สร้างกราฟด้วย Cosine Similarity
similarity_matrix = cosine_similarity(tfidf_matrix)
nx_graph = nx.from_numpy_array(similarity_matrix)

# ใช้ PageRank เพื่อจัดอันดับ
scores = nx.pagerank(nx_graph)
ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)

# แสดงผลสรุป (เลือก 2 ประโยคสำคัญ)
summary = ". ".join([ranked_sentences[i][1] for i in range(2)])
print("Summary:", summary)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

# ข้อความต้นฉบับและ Labels (1 = ประโยคสำคัญ, 0 = ไม่สำคัญ)
sentences = [
    "Coffee originated in Yemen in the 15th century.",
    "It was used by Sufi circles for religious rituals.",
    "From Yemen, coffee spread to the Middle East, Europe, and eventually the Americas.",
    "Today, coffee is one of the most popular beverages in the world."
]
labels = [1, 0, 1, 0]  # กำหนด labels ด้วยมือสำหรับตัวอย่างนี้

# เปลี่ยนข้อความเป็นเวกเตอร์
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(sentences)

# ฝึกโมเดล
clf = RandomForestClassifier()
clf.fit(X, labels)

# ทำนายความสำคัญของประโยค
predictions = clf.predict(X)

# เลือกประโยคสำคัญ
summary = ". ".join([sentences[i] for i in range(len(predictions)) if predictions[i] == 1])
print("Summary:", summary)


In [ ]:
scores

array([5.6369092 , 4.95706802, 8.39824451, 5.6095672 , 5.82396022,
       5.08447651, 7.04961844, 4.36432911, 3.30868744, 4.32544122,
       3.00470479, 4.7102547 , 3.03467019])

In [ ]:
sentences

[' The earliest credible evidence of coffee drinking or knowledge of the coffee tree appears in the middle of the 15th century in the accounts of Ahmed al-Ghaffar in Yemen,\n[3] where coffee seeds were first roasted and brewed in a similar way to how it is prepared now',
 'Coffee was used by Sufi circles to stay awake for their religious rituals.\n[13] Accounts differ on the origin of the coffee plant before its appearance in Yemen',
 "From Ethiopia, coffee could have been introduced to Yemen via trade across the Red Sea.\n[14] One account credits Muhammad Ibn Sa'd for bringing the beverage to Aden from the African coast,[15] other early accounts say Ali ben Omar of the Shadhili Sufi order was the first to introduce coffee to Arabia.\n[15][16] 16th-century Islamic scholar Ibn Hajar al-Haytami notes in his writings that a beverage called qahwa developed from a tree in the Zeila region located in the Horn of Africa.\n[13] Coffee was first exported from Ethiopia to Yemen by Somali merchan

In [ ]:
ranked_sentences

["From Ethiopia, coffee could have been introduced to Yemen via trade across the Red Sea.\n[14] One account credits Muhammad Ibn Sa'd for bringing the beverage to Aden from the African coast,[15] other early accounts say Ali ben Omar of the Shadhili Sufi order was the first to introduce coffee to Arabia.\n[15][16] 16th-century Islamic scholar Ibn Hajar al-Haytami notes in his writings that a beverage called qahwa developed from a tree in the Zeila region located in the Horn of Africa.\n[13] Coffee was first exported from Ethiopia to Yemen by Somali merchants from Berbera and Zeila in modern-day Somaliland, which was procured from Harar and the Abyssinian interior",
 'The first plants from these smuggled seeds were planted in Mysore.\nCoffee had spread to Italy by 1600 and then to the rest of Europe, Indonesia, and the Americas.[19]In 1583, Leonhard Rauwolf, a German physician, gave this description of coffee after returning from a ten-year trip\nto the Near East: A beverage as black as

In [ ]:
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# ข้อความต้นฉบับ
text = """
Coffee originated in Yemen in the 15th century. It was used by Sufi circles for religious rituals.
From Yemen, coffee spread to the Middle East, Europe, and eventually the Americas.
Today, coffee is one of the most popular beverages in the world.
"""

# แยกข้อความเป็นประโยค
sentences = text.split(". ")

# คำนวณ TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(sentences)

# สร้างกราฟด้วย Cosine Similarity
similarity_matrix = cosine_similarity(tfidf_matrix)
nx_graph = nx.from_numpy_array(similarity_matrix)

# ใช้ PageRank เพื่อจัดอันดับ
scores = nx.pagerank(nx_graph)
ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)

# เลือก 2 ประโยคสำคัญ
selected_sentences = [ranked_sentences[i][1] for i in range(2)]
print("Selected Sentences:", selected_sentences)


In [ ]:
from transformers import pipeline

# รวมประโยคที่เลือก
input_text = ". ".join(selected_sentences)

# ใช้ T5 Summarization
summarizer = pipeline("summarization", model="t5-small")
summary = summarizer(input_text, max_length=50, min_length=10, do_sample=False)

print("Final Summary:", summary[0]["summary_text"])


In [ ]:
# ติดตั้งไลบรารีก่อน (ถ้ายังไม่มี)
# pip install nltk

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import Counter

# โหลดข้อมูล Stopwords (ใช้ครั้งแรกเท่านั้น)
nltk.download('punkt')
nltk.download('stopwords')

# ข้อความตัวอย่าง (ปรับเปลี่ยนตามต้องการ)
text = """
Natural Language Processing (NLP) is a field of artificial intelligence that focuses on the interaction between humans and computers using natural language. The ultimate objective of NLP is to enable computers to understand, interpret, and respond to human language in a way that is both meaningful and useful. Applications of NLP are diverse, ranging from text summarization to machine translation, sentiment analysis, and question-answering systems.
"""

# 1. แบ่งข้อความเป็นประโยค
sentences = sent_tokenize(text)

# 2. ลบ stopwords และนับความถี่ของคำ
stop_words = set(stopwords.words('english'))
words = word_tokenize(text.lower())
filtered_words = [word for word in words if word.isalnum() and word not in stop_words]
word_frequencies = Counter(filtered_words)

# 3. ให้คะแนนความสำคัญของประโยค (Sentence Scoring)
sentence_scores = {}
for sent in sentences:
    for word in word_tokenize(sent.lower()):
        if word in word_frequencies:
            if sent not in sentence_scores:
                sentence_scores[sent] = word_frequencies[word]
            else:
                sentence_scores[sent] += word_frequencies[word]

# 4. เลือกประโยคที่สำคัญที่สุด
import heapq
summary_sentences = heapq.nlargest(2, sentence_scores, key=sentence_scores.get)
summary = ' '.join(summary_sentences)

print("Original Text:")
print(text)
print("\nExtracted Summary:")
print(summary)
